In [1]:
import pandas as pd
import psycopg2
from psycopg2 import Error

In [2]:
try:
    # Connect to an existing database
    connection = psycopg2.connect(user="user",
                                password="batugin2",
                                host="34.101.98.206",
                                port="5432",
                                database="dwh-supplychain")

    # Create a cursor to perform database operations
    cursor = connection.cursor()
    # Print PostgreSQL details
    print("PostgreSQL server information")
    print(connection.get_dsn_parameters(), "\n")
    # Executing a SQL query
    cursor.execute("SELECT version();")
    # Fetch result
    record = cursor.fetchone()
    print("You are connected to - ", record, "\n")
except (Exception, Error) as error:
    print("Error while connecting to PostgreSQL", error)

cursor.execute('''SELECT plant, plant_description, posting_date, material, bill_qty_sku, material_desc FROM v_sales_2021''')
data = cursor.fetchall()

col = []
for elt in cursor.description:
    col.append(elt[0])

df=pd.DataFrame(data=data, columns=col)
df.info()
df.head()

PostgreSQL server information
{'user': 'user', 'channel_binding': 'prefer', 'dbname': 'dwh-supplychain', 'host': '34.101.98.206', 'port': '5432', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'sslsni': '1', 'ssl_min_protocol_version': 'TLSv1.2', 'gssencmode': 'disable', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 

You are connected to -  ('PostgreSQL 14.4 on x86_64-pc-linux-gnu, compiled by Debian clang version 12.0.1, 64-bit',) 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 841600 entries, 0 to 841599
Data columns (total 6 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   plant              841600 non-null  object
 1   plant_description  841600 non-null  object
 2   posting_date       841600 non-null  object
 3   material           841600 non-null  object
 4   bill_qty_sku       841600 non-null  object
 5   material_desc      841600 non-null  object
dtypes: object(6)
memory usage: 38.5+ MB


,plant,plant_description,posting_date,material,bill_qty_sku,material_desc
0,2216,KFTD Bandung,2021-03-27,11000385,96.00,BETASON CREAM (DUS 24 TUB @ 5 G)
1,2218,KFTD Bogor,2021-03-31,11000262,200.00,SALBUTAMOL 4 MG (DUS 100 TAB)
2,2218,KFTD Bogor,2021-03-31,11000113,500.00,NITROKAF RETARD KAPSUL (DUS 100 KPS)
3,2218,KFTD Bogor,2021-03-31,11000383,25.00,BETAMETASON 0.1% CR(DUS 25 TUBE@5 GRAM)
4,2218,KFTD Bogor,2021-03-31,11000113,100.00,NITROKAF RETARD KAPSUL (DUS 100 KPS)


In [3]:
df_column = list(df.columns)
df_column

['plant',
 'plant_description',
 'posting_date',
 'material',
 'bill_qty_sku',
 'material_desc']

In [4]:
import numpy as np
df['posting_date'] = pd.to_datetime(df['posting_date'])
df['material'] = df['material'].astype(np.int64)
df['bill_qty_sku'] = df['bill_qty_sku'].astype(np.int64)
df['plant'] = df['plant'].astype(np.int64)
df.info()
print(df['posting_date'].min())
print(df['posting_date'].max())
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 841600 entries, 0 to 841599
Data columns (total 6 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   plant              841600 non-null  int64         
 1   plant_description  841600 non-null  object        
 2   posting_date       841600 non-null  datetime64[ns]
 3   material           841600 non-null  int64         
 4   bill_qty_sku       841600 non-null  int64         
 5   material_desc      841600 non-null  object        
dtypes: datetime64[ns](1), int64(3), object(2)
memory usage: 38.5+ MB
2021-01-07 00:00:00
2021-05-31 00:00:00


,plant,plant_description,posting_date,material,bill_qty_sku,material_desc
0,2216,KFTD Bandung,2021-03-27,11000385,96,BETASON CREAM (DUS 24 TUB @ 5 G)
1,2218,KFTD Bogor,2021-03-31,11000262,200,SALBUTAMOL 4 MG (DUS 100 TAB)
2,2218,KFTD Bogor,2021-03-31,11000113,500,NITROKAF RETARD KAPSUL (DUS 100 KPS)
3,2218,KFTD Bogor,2021-03-31,11000383,25,BETAMETASON 0.1% CR(DUS 25 TUBE@5 GRAM)
4,2218,KFTD Bogor,2021-03-31,11000113,100,NITROKAF RETARD KAPSUL (DUS 100 KPS)
...,...,...,...,...,...,...
841595,2203,KFTD Pematangsiantar,2021-02-27,11000113,1000,NITROKAF RETARD KAPSUL (DUS 100 KPS)
841596,2203,KFTD Pematangsiantar,2021-02-27,11000113,3000,NITROKAF RETARD KAPSUL (DUS 100 KPS)
841597,2203,KFTD Pematangsiantar,2021-02-27,11000257,4000,RETAPHYL SR-STRIP 10X10 TABLET
841598,2203,KFTD Pematangsiantar,2021-02-27,11000391,100,GENTAMYCIN 0.1 % (DUS 10 TUBE @ 5 GRAM)


In [7]:
material_list = df.groupby(['material'], as_index=False).agg({'material':'first','material_desc':'first'})
material_list.info()
material_list.reset_index(drop=True)
material_list.to_csv("material_list.csv")
material_list.to_excel("material_list.xlsx")
material_list

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 549 entries, 0 to 548
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   material       549 non-null    int64 
 1   material_desc  549 non-null    object
dtypes: int64(1), object(1)
memory usage: 8.7+ KB


,material,material_desc
0,11000000,ACICLOVIR 200 MG (DUS 100 TAB)
1,11000002,ACICLOVIR 400 MG (DUS 100 TAB)
2,11000004,ACICLOVIR CREAM 5%(DUS 25 TUBE @ 5 GRAM)
3,11000006,ALERGINE 10 MG TSS (DUS 50 TAB)
4,11000008,ALPRAZOLAM 0.5 MG (DUS 100 TAB)
...,...,...
544,11002173,VINBLASTINE SULPHATE 10MG INJ
545,11002174,"CABERGOLINE TAB 0,25MG @8"
546,11002182,SUBOXONE TABLET 8 MG (DUS 7 TAB)
547,11002198,FLUORESCEIN SODIUM 20% 3ML


In [6]:
material_list = pd.read_csv('E:/Program/PKLProjekt/Jupyter/Sales-Forecasting/material_list.csv')
material_list = material_list.drop(['material_desc'],axis=1)
material_clean = pd.DataFrame()
date_range = pd.date_range(start='2021-01-07', end='2021-05-31', freq='D')
for i in material_list['material'].unique():
    current = df.loc[(df['material']==i)].copy()
    if current.shape[0] < 90:
        print('Data material: ', i, 'tidak mencukupi')
    else:
        current = current.groupby(['posting_date']).agg({'material':'first','bill_qty_sku':'sum'})
        current.resample('D')
        current = current.reindex(date_range)
        current['bill_qty_sku'] = current['bill_qty_sku'].fillna(0)
        current['material'] = current['material'].fillna(i)
        current = current.reset_index()
        current['bill_qty_sku'] = current['bill_qty_sku'].astype(np.int64)
        current['material'] = current['material'].astype(np.int64)
        current = current.rename(columns={'index':'date'})
        material_clean = pd.concat([material_clean, current], axis=0)

material_clean.info()
material_clean.to_csv('E:/Program/PKLProjekt/Jupyter/Sales-Forecasting/material_df.csv')
material_clean.to_excel('E:/Program/PKLProjekt/Jupyter/Sales-Forecasting/material_df.xlsx')
material_clean

Data material:  11000013 tidak mencukupi
Data material:  11000017 tidak mencukupi
Data material:  11000020 tidak mencukupi
Data material:  11000024 tidak mencukupi
Data material:  11000050 tidak mencukupi
Data material:  11000060 tidak mencukupi
Data material:  11000078 tidak mencukupi
Data material:  11000088 tidak mencukupi
Data material:  11000089 tidak mencukupi
Data material:  11000091 tidak mencukupi
Data material:  11000092 tidak mencukupi
Data material:  11000117 tidak mencukupi
Data material:  11000118 tidak mencukupi
Data material:  11000133 tidak mencukupi
Data material:  11000138 tidak mencukupi
Data material:  11000143 tidak mencukupi
Data material:  11000148 tidak mencukupi
Data material:  11000183 tidak mencukupi
Data material:  11000185 tidak mencukupi
Data material:  11000189 tidak mencukupi
Data material:  11000207 tidak mencukupi
Data material:  11000238 tidak mencukupi
Data material:  11000256 tidak mencukupi
Data material:  11000260 tidak mencukupi
Data material:  

,date,material,bill_qty_sku
0,2021-01-07,11000000,100
1,2021-01-08,11000000,1900
2,2021-01-09,11000000,2700
3,2021-01-10,11000000,0
4,2021-01-11,11000000,5700
...,...,...,...
140,2021-05-27,11002125,0
141,2021-05-28,11002125,0
142,2021-05-29,11002125,0
143,2021-05-30,11002125,0


In [7]:
train_start = '2021-01-07'
train_end = '2021-04-30'
test_start = '2021-05-01'
test_end = '2021-05-31'
train_df = material_clean.loc[(material_clean['date'] >= train_start) & (material_clean['date'] <= train_end)]
test_df = material_clean.loc[(material_clean['date'] >= test_start) & (material_clean['date'] <= test_end)]

In [8]:
test_df.to_csv('E:/Program/PKLProjekt/Jupyter/Sales-Forecasting/test_df.csv')

In [9]:
train_df.to_csv('E:/Program/PKLProjekt/Jupyter/Sales-Forecasting/train_df.csv')

In [ ]:
date_range = pd.date_range(start='2021-01-07', end='2021-05-31', freq='D')
current = df.loc[(df['material']==11000002)].copy()
current = current.groupby(['posting_date']).agg({'material':'first','bill_qty_sku':'sum'})
current.resample('D')
current = current.reindex(date_range)
current['bill_qty_sku'] = current['bill_qty_sku'].fillna(0)
current['material'] = current['material'].fillna(method='ffill')
current = current.reset_index()
current.rename(columns={'index':'date'})
current['bill_qty_sku'] = current['bill_qty_sku'].astype(np.int64)
current['material'] = current['material'].astype(np.int64)
current = current.rename(columns={'index':'date'})
current